In [ ]:
import Utilities as U
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X,y=U.load_dataset(.33)
X1=X.drop(X.columns[np.std(X)==0.0],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X1,y,test_size=.20)

c:\Users\schio\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
from sklearn import svm
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
cl = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)


In [3]:
from sklearn.metrics import accuracy_score


print(accuracy_score(y_test,cl.predict(X_test)))

0.9341428571428572
